# Übung 10: Widgets
Für die folgenden Aufgaben benötigst du die Datei <font color=orange>bigdata/120-years-of-olympic-history-athletes-and-results/athlete_events.csv</font>. Importiere sie entsprechend.

In [ ]:
import pandas as pd
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

In [ ]:
df = pd.read_csv('../../src/bigdata/120-years-of-olympic-history-athletes-and-results/athlete_events.csv')
df.head()

# Teilaufgabe 1
Erstelle mithilfe von ``@interact`` eine Funktion, die dir nur die Daten aus einen bestimmten Jahr anzeigt. Das Jahr soll mit einem **Slider** ausgewählt werden können. Damit der DataFrame nicht so groß wird filtere vorher die Daten, sodass Sie nur die Daten deiner Liebelingssportart enthalten (z.B. 'Table Tennis' oder 'Beach Volleyball'). 


**Verbesserung des Sliders, wenn du mit der obigen Aufgabe fertig bist**:<br>
Die olympischen Spiele finden nur alle zwei Jahre statt. Passe den **Slider** zur Auswahl des Jahres so an, das dieser in 4 Jahres Schritten (Sommerspiele) oder in 2 Jahres Schritten (Winterspiele, früher alle 2 Jahre) arbeitet. Außerdem überlege dir eine sinnvolle grobsortierung des Ergebnisses.

In [ ]:
# Wähle eine Sportart aus
df['Sport'].unique()

In [ ]:
# Filtern der Sportart
df_my_sport = df[df['Sport'] == 'Table Tennis']
df_my_sport.head()

In [ ]:
print('Obere und untere Grenze bestimmen')
# ----- Lösung -----

# Bestimmen der oberen und der unteren Grenze
print(df_my_sport['Year'].min(), df_my_sport['Year'].max())

In [ ]:
print('Automatische Grenzen mit Slider')
# ----- Lösung -----

# Die Minimal und Maximal Werte für den Slider können wir uns durch den DataFrame bestimmen lassen
@interact
def year_single(year=(df_my_sport['Year'].min(), df_my_sport['Year'].max())):
    df_filtered = df_my_sport[df_my_sport['Year'] == year]
    return df_filtered.sort_values(['Year'])

In [ ]:
# Sommerspiele
print('Sommerspiele')
# ----- Lösung -----

@interact
def year_single(year=(df_my_sport['Year'].min(), df_my_sport['Year'].max(), 4)):
    df_filtered = df_my_sport[df_my_sport['Year'] == year]
    return df_filtered.sort_values(['Year'])

In [ ]:
# Winterspiele
print('Winterspiele')
# ----- Lösung -----

df_my_sport = df[df['Sport'] == 'Ski Jumping']
@interact
def year_single(year=(df_my_sport['Year'].min(), df_my_sport['Year'].max(), 2)):
    df_filtered = df_my_sport[df_my_sport['Year'] == year]
    return df_filtered.sort_values(['Year'])

## Teilaufgabe 2
Schreibe ein interative Funktion mit derer Hilfe du nach ihrem Namen suchen kannst um die ID des Athleten zu erhalten.

In [ ]:
@interact
def search_athlet(name=''):
    df_reduced = df[['ID', 'Name']].drop_duplicates() # Wir brauchen jeden Athleten nur ein Mal
    return df_reduced[df_reduced['Name'].str.contains(name)]

## Teilaufgabe 3
Filter den DataFrame nach Datensätzen einer bestimmten Sportart. Lasse dir hierfür ein Dropdown mit allen Sportarten aus der Spalte Sport generieren.

<font color="red">**Warnung:** Generiere die Liste aller einzigartigen Sportarten und speichere Sie vorher in einer Liste. Solltes du versuchen die ganze Spalte als dropdown zu übergeben kann es sein, dass dein Notebook abstürzt.</font>

In [ ]:
# Überprüfen der Sportarten
stypes=df['Sport'].sort_values().unique()
stypes

In [ ]:

# type ist ein reserviertes Wort daher können wir hier nur ein anderes Wort benutzen, z.B. stype 
@interact
def sport_type(stype=stypes):
    return df[df['Sport'] == stype].sort_values('Year')

## Zusatzübung 1:
In der Gruppieren Lektion wurde gezeigt, wie man 

In [ ]:
df_teilnehmer = df.drop_duplicates(['ID', 'Year'])
df_teilnehmer = df_teilnehmer[df_teilnehmer['Season'] == 'Summer']
df_teilnehmer_mw = df_teilnehmer.groupby(['Year', 'Sex']).size()
df_teilnehmer_mw = df_teilnehmer_mw.unstack()
df_teilnehmer_mw.head()

In [ ]:
@interact
def teilnehmer_mw(Geschlecht=['F', 'M']):
    return df_teilnehmer_mw[[Geschlecht]].plot()

## Zusatzübung 2:
Probiere weitere Optionen aus indem du dir die Liste der verfügbaren Widgets anschaust. Löse die folgenden Darstellungen mit einem **einzigen Widget**.

Link zu den Widgets: https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html

* Lasse dir nur Athleten anzeigen die eine bestimme Anzahl an Goldmedalien in den Jahren 2000-2016 gewonnen haben. Sorge dafür, das sowohl die Ober- als auch die Untergrenze angepasst werden kann und nur Athleten in der Auswahl erscheinen, die eine Anzahl an Goldmedallien innerhalb dieser Grenzen gewonnen haben.

*Tipp: Schau dir die Funktion* **.between()** *in der offiziellen Pandas Documentation an*



In [ ]:
# Vorbereitung des DataFrames
df_medals = df.dropna(subset=['Medal'])
df_medals = df_medals[df_medals['Medal'] == 'Gold']
df_medals = df_medals[df_medals['Year'].between(2000, 2016)]
df_medals = df_medals.groupby(['ID', 'Name']).size().reset_index()
df_medals = df_medals.rename(columns={0:'Count'})
df_medals.head()

In [ ]:
widgets.IntRangeSlider(min=df_medals['Count'].min(), max=df_medals['Count'].max())

In [ ]:
# Grenzen lassen wir uns automatisch berechnen
sw = widgets.IntRangeSlider(min=df_medals['Count'].min(), max=df_medals['Count'].max())

# Das Ergebnis aus sw ist eine Liste mit zwei Elementen. Das erste Element enthält 
# die untere Grenze wohingegen das zweite Element die obere Grenze enthält.

@interact
def medals_count(count=sw):
    return df_medals[df_medals['Count'].between(count[0], count[1])].sort_values('Count', ascending=False)

## Zusatzaufgabe 3
Es soll nun möglich sein nur eine Liste alle Athleten und die Anzahl Ihrer Medalien abhänig von einem durch zwei **DatePicker** gewählte Daten zu betrachten. 

*Tipp: Für die Verarbeitung der Daten kann die Lösung aus der vorangegangen Gruppierungs Übung benutzt werden.*

In [ ]:
# Keine Medaillen ist noch mit NaN ausgedrückt, hier wollen wir einen schöneren Begriff haben.
df_medals = df.fillna('Teilnahme')
df_medals = df_medals.groupby(['ID', 'Name', 'Year', 'Medal']).size()
df_medals = df_medals.unstack()
df_medals = df_medals.fillna(0)
df_medals = df_medals.reset_index()
df_medals.head()

In [ ]:
# Achtung: Year ist eine Zahl und das Ergebnis von pd.to_datetime datetime.
dp_start = widgets.DatePicker(value=pd.to_datetime('1896-01-01'))
dp_end = widgets.DatePicker(value=pd.to_datetime('2019-01-01'))

@interact
def start_end(start_year=dp_start, end_year=dp_end):
    return df_medals[(df_medals['Year'] >= start_year.year) & (df_medals['Year'] <= end_year.year)].sort_values('Year')